In [1]:
from PIL import Image
import numpy as np
from skimage import segmentation
import matplotlib.pyplot as plt

def calculate_color_score(image_path, mask_path):
    # Load the image and its corresponding mask
    image = Image.open(image_path)
    mask = Image.open(mask_path).convert('L')

    # Convert PIL image to numpy array
    rgb_img = np.array(image)
    mask = np.array(mask)

    # Find coordinates of the lesion in the mask
    lesion_coords = np.where(mask != 0)
    min_x = min(lesion_coords[0])
    max_x = max(lesion_coords[0])
    min_y = min(lesion_coords[1])
    max_y = max(lesion_coords[1])
    cropped_lesion = rgb_img[min_x:max_x, min_y:max_y]

    # Perform SLIC superpixel segmentation
    labels1 = segmentation.slic(cropped_lesion, compactness=10, n_segments=10, sigma=3,
                                start_label=1)

    # Calculate average color for each segment
    segment_colors = []
    for label in np.unique(labels1):
        if label == 0:
            continue
        mask_label = labels1 == label
        avg_color = np.mean(cropped_lesion[mask_label], axis=0)
        segment_colors.append(avg_color)

    # Calculate average color of all colors in the lesion
    avg_lesion_color = np.mean(cropped_lesion, axis=(0, 1))

    # Calculate the variance of colors within the lesion
    color_variance = np.var(cropped_lesion, axis=(0, 1))

    # Compute the colorfulness score as the sum of variances across all channels
    colorfulness_score = np.sum(color_variance)

    # Categorize the colorfulness score 
    if colorfulness_score > 10000:
        color_score = 4
    elif colorfulness_score > 5000:
        color_score = 3
    elif colorfulness_score > 1000:
        color_score = 2
    else:
        color_score = 1

    return color_score

#add image path and calclate the lesion color 
image_path = " "
mask_path = " "

color_score = calculate_color_score(image_path, mask_path)
print("Lesion Color Score:", color_score)


Lesion Color Score: 1
